# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 02 · Notebook 6 — Advanced Dashboard Features: Production-Ready Applications
**Instructor:** Amir Charkhi  |  **Goal:** Master advanced features for professional dashboards.

> Format: real-time updates → downloads → authentication → responsive design → integrations.


---
## From Basic to Professional
Transform your dashboards with advanced features that users expect in production applications.

## Setup and Dependencies

In [ ]:
# Install required packages
import subprocess
import sys
import os

# List of required packages
packages = [
    'streamlit',
    'pandas',
    'numpy',
    'plotly',
    'requests',
    'openpyxl',
    'xlsxwriter'
]

# Install packages if needed
for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("\nAll packages installed successfully!")

## Important Notes

**Remember:** Streamlit apps run as Python scripts, not in Jupyter notebooks.

Each code example creates a `.py` file that you run with:
```bash
streamlit run filename.py
```

We'll build increasingly complex features throughout this module.

## 1. Real-Time Data Updates

Learn how to create dashboards that update automatically with new data.

### 1.1 Auto-Refresh Dashboard

In [ ]:
# Create auto-refreshing dashboard
code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import time

st.set_page_config(
    page_title="Real-Time Dashboard",
    layout="wide"
)

st.title("Real-Time Data Dashboard")
st.markdown("Dashboard updates automatically every few seconds")

# Sidebar controls
st.sidebar.header("Settings")
auto_refresh = st.sidebar.checkbox("Auto Refresh", value=True)
refresh_interval = st.sidebar.slider("Refresh Interval (seconds)", 1, 10, 3)

# Placeholder for metrics
placeholder = st.empty()

# Function to generate random data
def get_current_data():
    return {
        "timestamp": datetime.now(),
        "users_online": np.random.randint(100, 500),
        "cpu_usage": np.random.uniform(20, 80),
        "memory_usage": np.random.uniform(30, 70),
        "requests_per_sec": np.random.randint(50, 200),
        "error_rate": np.random.uniform(0, 5)
    }

# Initialize session state for historical data
if "history" not in st.session_state:
    st.session_state.history = []

# Main update loop
if auto_refresh:
    for _ in range(10):
        with placeholder.container():
            # Get current data
            current_data = get_current_data()
            
            # Add to history
            st.session_state.history.append(current_data)
            
            # Keep only last 100 points
            if len(st.session_state.history) > 100:
                st.session_state.history = st.session_state.history[-100:]
            
            # Display metrics
            col1, col2, col3, col4, col5 = st.columns(5)
            
            with col1:
                st.metric(
                    "Users Online",
                    current_data["users_online"],
                    delta=np.random.randint(-10, 20)
                )
            
            with col2:
                st.metric(
                    "CPU Usage",
                    f"{current_data['cpu_usage']:.1f}%",
                    delta=f"{np.random.uniform(-5, 5):.1f}%"
                )
            
            with col3:
                st.metric(
                    "Memory",
                    f"{current_data['memory_usage']:.1f}%",
                    delta=f"{np.random.uniform(-3, 3):.1f}%"
                )
            
            with col4:
                st.metric(
                    "Requests/sec",
                    current_data["requests_per_sec"],
                    delta=np.random.randint(-20, 30)
                )
            
            with col5:
                st.metric(
                    "Error Rate",
                    f"{current_data['error_rate']:.2f}%",
                    delta=f"{np.random.uniform(-0.5, 0.5):.2f}%"
                )
            
            # Create real-time chart
            if len(st.session_state.history) > 1:
                df = pd.DataFrame(st.session_state.history)
                
                fig = go.Figure()
                
                fig.add_trace(go.Scatter(
                    x=df["timestamp"],
                    y=df["cpu_usage"],
                    mode="lines",
                    name="CPU Usage",
                    line=dict(color="blue")
                ))
                
                fig.add_trace(go.Scatter(
                    x=df["timestamp"],
                    y=df["memory_usage"],
                    mode="lines",
                    name="Memory Usage",
                    line=dict(color="green")
                ))
                
                fig.update_layout(
                    title="System Metrics Over Time",
                    xaxis_title="Time",
                    yaxis_title="Usage (%)",
                    height=400
                )
                
                st.plotly_chart(fig, use_container_width=True)
            
            # Display last update time
            st.caption(f"Last updated: {current_data['timestamp'].strftime('%H:%M:%S')}")
        
        time.sleep(refresh_interval)
        
        if not auto_refresh:
            break
'''

with open('realtime_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created realtime_dashboard.py")
print("Run with: streamlit run realtime_dashboard.py")

## 2. Download Functionality

Enable users to export data and reports in various formats.

In [ ]:
# Create dashboard with download functionality
code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import io
from datetime import datetime
import json

st.set_page_config(
    page_title="Export Dashboard",
    layout="wide"
)

st.title("Data Export Dashboard")
st.markdown("Export your data in multiple formats")

# Generate sample data
@st.cache_data
def generate_data():
    np.random.seed(42)
    dates = pd.date_range(start="2023-01-01", periods=100, freq="D")
    
    data = pd.DataFrame({
        "date": dates,
        "sales": np.random.uniform(1000, 5000, 100),
        "costs": np.random.uniform(500, 2500, 100),
        "customers": np.random.randint(50, 200, 100),
        "region": np.random.choice(["North", "South", "East", "West"], 100),
        "product": np.random.choice(["A", "B", "C"], 100)
    })
    
    data["profit"] = data["sales"] - data["costs"]
    data["margin"] = (data["profit"] / data["sales"]) * 100
    
    return data

df = generate_data()

# Sidebar filters
st.sidebar.header("Filters")
selected_regions = st.sidebar.multiselect(
    "Select Regions",
    options=df["region"].unique(),
    default=df["region"].unique()
)

selected_products = st.sidebar.multiselect(
    "Select Products",
    options=df["product"].unique(),
    default=df["product"].unique()
)

# Apply filters
filtered_df = df[
    (df["region"].isin(selected_regions)) &
    (df["product"].isin(selected_products))
]

# Display metrics
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric("Total Sales", f"${filtered_df['sales'].sum():,.0f}")
with col2:
    st.metric("Total Profit", f"${filtered_df['profit'].sum():,.0f}")
with col3:
    st.metric("Avg Margin", f"{filtered_df['margin'].mean():.1f}%")
with col4:
    st.metric("Total Customers", f"{filtered_df['customers'].sum():,}")

# Display chart
st.subheader("Sales Trend")
fig = px.line(filtered_df, x="date", y="sales", color="region")
st.plotly_chart(fig, use_container_width=True)

# Download section
st.markdown("---")
st.subheader("Export Data")

col1, col2, col3 = st.columns(3)

# CSV Download
with col1:
    csv = filtered_df.to_csv(index=False)
    st.download_button(
        label="Download CSV",
        data=csv,
        file_name=f"data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
        mime="text/csv"
    )

# Excel Download
with col2:
    buffer = io.BytesIO()
    with pd.ExcelWriter(buffer, engine="xlsxwriter") as writer:
        filtered_df.to_excel(writer, sheet_name="Data", index=False)
    
    st.download_button(
        label="Download Excel",
        data=buffer.getvalue(),
        file_name=f"report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

# JSON Download
with col3:
    json_data = filtered_df.to_json(orient="records", date_format="iso")
    st.download_button(
        label="Download JSON",
        data=json_data,
        file_name=f"data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
        mime="application/json"
    )

# Display data table
st.markdown("---")
st.subheader("Data Preview")
st.dataframe(filtered_df, use_container_width=True)
'''

with open('download_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created download_dashboard.py")
print("Run with: streamlit run download_dashboard.py")

## 3. User Authentication Basics

Implement simple authentication to protect your dashboards.

In [ ]:
# Create authentication dashboard
code = '''
import streamlit as st
import pandas as pd
import hashlib
from datetime import datetime
import numpy as np

st.set_page_config(
    page_title="Secure Dashboard",
    layout="wide"
)

# Simple hash function for passwords
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# User database (in production, use a real database)
USERS = {
    "admin": {
        "password": hash_password("admin123"),
        "role": "admin",
        "name": "Administrator"
    },
    "user": {
        "password": hash_password("user123"),
        "role": "user",
        "name": "Regular User"
    },
    "viewer": {
        "password": hash_password("viewer123"),
        "role": "viewer",
        "name": "View Only User"
    }
}

# Initialize session state
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False
if "username" not in st.session_state:
    st.session_state.username = None
if "role" not in st.session_state:
    st.session_state.role = None

# Login function
def login(username, password):
    if username in USERS:
        if USERS[username]["password"] == hash_password(password):
            st.session_state.authenticated = True
            st.session_state.username = username
            st.session_state.role = USERS[username]["role"]
            st.session_state.name = USERS[username]["name"]
            return True
    return False

# Logout function
def logout():
    st.session_state.authenticated = False
    st.session_state.username = None
    st.session_state.role = None
    st.session_state.name = None

# Login page
if not st.session_state.authenticated:
    st.title("Login")
    
    col1, col2, col3 = st.columns([1, 2, 1])
    
    with col2:
        with st.form("login_form"):
            st.markdown("### Enter your credentials")
            username = st.text_input("Username")
            password = st.text_input("Password", type="password")
            
            if st.form_submit_button("Login", type="primary", use_container_width=True):
                if login(username, password):
                    st.success("Login successful!")
                    st.rerun()
                else:
                    st.error("Invalid username or password")
        
        # Demo credentials
        with st.expander("Demo Credentials"):
            st.markdown("""
            **Admin User:**
            - Username: admin
            - Password: admin123
            
            **Regular User:**
            - Username: user
            - Password: user123
            
            **Viewer:**
            - Username: viewer
            - Password: viewer123
            """)

else:
    # Main dashboard (only visible after login)
    st.title("Secure Dashboard")
    
    # User info and logout
    col1, col2, col3 = st.columns([2, 2, 1])
    with col1:
        st.markdown(f"**Welcome, {st.session_state.name}!**")
    with col2:
        st.markdown(f"**Role:** {st.session_state.role.title()}")
    with col3:
        if st.button("Logout", type="secondary"):
            logout()
            st.rerun()
    
    st.markdown("---")
    
    # Role-based content
    if st.session_state.role == "admin":
        st.subheader("Admin Dashboard")
        
        # Admin metrics
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("Total Users", "1,234")
        with col2:
            st.metric("Active Sessions", "42")
        with col3:
            st.metric("Revenue", "$125,430")
        with col4:
            st.metric("System Health", "98%")
        
        # Sample chart
        chart_data = pd.DataFrame(
            np.random.randn(20, 3),
            columns=["Sales", "Revenue", "Growth"]
        )
        st.line_chart(chart_data)
    
    elif st.session_state.role == "user":
        st.subheader("User Dashboard")
        
        # Limited functionality for regular users
        col1, col2 = st.columns(2)
        
        with col1:
            st.metric("Your Tasks", "12")
            st.metric("Completed", "8")
        
        with col2:
            st.metric("Pending", "4")
            st.metric("Performance", "85%")
    
    else:  # viewer role
        st.subheader("Viewer Dashboard")
        st.info("You have read-only access. Contact an administrator for additional permissions.")
        
        # Read-only view
        st.markdown("### Public Reports")
        st.metric("Total Reports", "25")
'''

with open('auth_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created auth_dashboard.py")
print("Run with: streamlit run auth_dashboard.py")
print("\nDemo credentials:")
print("  Admin: admin/admin123")
print("  User: user/user123")
print("  Viewer: viewer/viewer123")

## 4. Responsive Design for Mobile

Create dashboards that work well on all screen sizes.

In [ ]:
# Create responsive dashboard
code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

st.set_page_config(
    page_title="Responsive Dashboard",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Custom CSS for responsive design
st.markdown("""
<style>
    @media (max-width: 768px) {
        .block-container {
            padding: 1rem;
        }
        
        .row-widget.stHorizontal {
            flex-direction: column;
        }
        
        h1 {
            font-size: 1.5rem;
        }
        
        h2 {
            font-size: 1.25rem;
        }
    }
</style>
""", unsafe_allow_html=True)

# Header
st.title("Responsive Dashboard")
st.caption("Optimized for all screen sizes")

# Mobile-friendly tabs instead of sidebar on small screens
tab1, tab2, tab3, tab4 = st.tabs(["Overview", "Analytics", "Data", "Settings"])

with tab1:
    st.header("Overview")
    
    # Responsive metrics grid
    col1, col2 = st.columns(2)
    with col1:
        st.metric("Revenue", "$45.2K", "+12%")
        st.metric("Users", "1,234", "+5%")
    with col2:
        st.metric("Orders", "342", "+8%")
        st.metric("Avg. Value", "$132", "-2%")
    
    # Responsive chart
    st.subheader("Trend")
    
    # Generate sample data
    dates = pd.date_range(start="2024-01-01", periods=30)
    data = pd.DataFrame({
        "Date": dates,
        "Value": np.random.randn(30).cumsum() + 100
    })
    
    fig = px.line(data, x="Date", y="Value", title="30-Day Trend")
    fig.update_layout(
        height=300,
        margin=dict(l=0, r=0, t=30, b=0),
        showlegend=False
    )
    st.plotly_chart(fig, use_container_width=True)

with tab2:
    st.header("Analytics")
    
    analysis_type = st.selectbox(
        "Select Analysis",
        ["Sales by Region", "Product Performance", "Customer Segments"]
    )
    
    if analysis_type == "Sales by Region":
        region_data = pd.DataFrame({
            "Region": ["North", "South", "East", "West"],
            "Sales": [45000, 38000, 52000, 41000]
        })
        
        fig = px.bar(
            region_data,
            x="Sales",
            y="Region",
            orientation="h",
            title="Sales by Region"
        )
        fig.update_layout(height=250, margin=dict(l=0, r=0, t=30, b=0))
        st.plotly_chart(fig, use_container_width=True)

with tab3:
    st.header("Data Table")
    
    # Generate sample data
    df = pd.DataFrame({
        "Date": pd.date_range(start="2024-01-01", periods=10),
        "Sales": np.random.randint(1000, 5000, 10),
        "Orders": np.random.randint(10, 50, 10),
        "Region": np.random.choice(["North", "South", "East", "West"], 10)
    })
    
    # Mobile-optimized table display
    display_columns = st.multiselect(
        "Select columns to display",
        options=df.columns.tolist(),
        default=["Date", "Sales"]
    )
    
    if display_columns:
        st.dataframe(
            df[display_columns],
            use_container_width=True,
            hide_index=True
        )
    
    # Download button
    csv = df.to_csv(index=False)
    st.download_button(
        label="Download CSV",
        data=csv,
        file_name="data.csv",
        mime="text/csv",
        use_container_width=True
    )

with tab4:
    st.header("Settings")
    
    with st.form("settings_form"):
        st.subheader("Display Preferences")
        
        theme = st.selectbox("Theme", ["Light", "Dark", "Auto"])
        notifications = st.checkbox("Enable notifications")
        
        submitted = st.form_submit_button(
            "Save Settings",
            type="primary",
            use_container_width=True
        )
        
        if submitted:
            st.success("Settings saved successfully!")
'''

with open('responsive_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created responsive_dashboard.py")
print("Run with: streamlit run responsive_dashboard.py")
print("\nTest on different screen sizes:")
print("  - Resize your browser window")
print("  - Use browser developer tools device emulation")
print("  - Test on actual mobile devices")

## 5. Integration with Databases and APIs

Connect your dashboards to real data sources.

### 5.1 Database Integration

In [ ]:
# Create database integration example
code = '''
import streamlit as st
import pandas as pd
import sqlite3
from datetime import datetime
import numpy as np

st.set_page_config(
    page_title="Database Dashboard",
    layout="wide"
)

st.title("Database Integration Dashboard")

# Create or connect to SQLite database
@st.cache_resource
def init_database():
    conn = sqlite3.connect("dashboard.db")
    cursor = conn.cursor()
    
    # Create table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS sales (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date DATE,
            product TEXT,
            region TEXT,
            amount REAL,
            quantity INTEGER
        )
    """)
    
    # Check if table is empty and add sample data
    cursor.execute("SELECT COUNT(*) FROM sales")
    if cursor.fetchone()[0] == 0:
        # Add sample data
        from datetime import timedelta
        
        products = ["Product A", "Product B", "Product C"]
        regions = ["North", "South", "East", "West"]
        
        for i in range(100):
            date = datetime.now() - timedelta(days=100-i)
            product = np.random.choice(products)
            region = np.random.choice(regions)
            amount = np.random.uniform(100, 1000)
            quantity = np.random.randint(1, 20)
            
            cursor.execute(
                "INSERT INTO sales (date, product, region, amount, quantity) VALUES (?, ?, ?, ?, ?)",
                (date.date(), product, region, amount, quantity)
            )
        
        conn.commit()
    
    return conn

# Initialize database
conn = init_database()

# Sidebar for database operations
st.sidebar.header("Database Operations")

operation = st.sidebar.selectbox(
    "Select Operation",
    ["View Data", "Add Record", "Query Builder"]
)

if operation == "View Data":
    st.header("View Database Tables")
    
    # Fetch data from database
    query = "SELECT * FROM sales ORDER BY id DESC LIMIT 100"
    df = pd.read_sql_query(query, conn)
    
    # Display metrics
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.metric("Total Records", len(df))
    with col2:
        st.metric("Total Sales", f"${df['amount'].sum():,.2f}")
    with col3:
        st.metric("Avg Order", f"${df['amount'].mean():,.2f}")
    
    # Display data
    st.dataframe(df, use_container_width=True)

elif operation == "Add Record":
    st.header("Add New Record")
    
    with st.form("add_sale"):
        col1, col2 = st.columns(2)
        
        with col1:
            date = st.date_input("Date")
            product = st.selectbox("Product", ["Product A", "Product B", "Product C"])
            region = st.selectbox("Region", ["North", "South", "East", "West"])
        
        with col2:
            amount = st.number_input("Amount", min_value=0.0, step=0.01)
            quantity = st.number_input("Quantity", min_value=1, step=1)
        
        if st.form_submit_button("Add Sale", type="primary"):
            cursor = conn.cursor()
            cursor.execute(
                "INSERT INTO sales (date, product, region, amount, quantity) VALUES (?, ?, ?, ?, ?)",
                (date, product, region, amount, quantity)
            )
            conn.commit()
            st.success("Sale record added successfully!")

elif operation == "Query Builder":
    st.header("SQL Query Builder")
    
    query = st.text_area(
        "Enter SQL Query",
        value="SELECT * FROM sales WHERE amount > 500 LIMIT 10",
        height=100
    )
    
    if st.button("Execute Query", type="primary"):
        try:
            result_df = pd.read_sql_query(query, conn)
            st.success(f"Query executed successfully! {len(result_df)} rows returned.")
            st.dataframe(result_df, use_container_width=True)
        except Exception as e:
            st.error(f"Error executing query: {str(e)}")
'''

with open('database_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created database_dashboard.py")
print("Run with: streamlit run database_dashboard.py")

### 5.2 API Integration

In [ ]:
# Create API integration dashboard
code = '''
import streamlit as st
import pandas as pd
import requests
import plotly.express as px

st.set_page_config(
    page_title="API Integration Dashboard",
    layout="wide"
)

st.title("API Integration Dashboard")
st.markdown("Connect to external data sources via APIs")

# Cache API calls
@st.cache_data(ttl=300)  # Cache for 5 minutes
def fetch_api_data(url, params=None):
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"API Error: {str(e)}")
        return None

# Weather API Example
st.header("Weather Data Dashboard")

col1, col2 = st.columns(2)

with col1:
    latitude = st.number_input("Latitude", value=40.7128, format="%.4f")
with col2:
    longitude = st.number_input("Longitude", value=-74.0060, format="%.4f")

if st.button("Fetch Weather Data", type="primary"):
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m,precipitation",
        "forecast_days": 3
    }
    
    with st.spinner("Fetching weather data..."):
        data = fetch_api_data("https://api.open-meteo.com/v1/forecast", params)
    
    if data:
        # Process weather data
        hourly_data = data.get("hourly", {})
        
        df = pd.DataFrame({
            "time": pd.to_datetime(hourly_data.get("time", [])),
            "temperature": hourly_data.get("temperature_2m", []),
            "precipitation": hourly_data.get("precipitation", [])
        })
        
        # Display metrics
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Current Temp", f"{df['temperature'].iloc[0]:.1f}°C")
        with col2:
            st.metric("Max Temp", f"{df['temperature'].max():.1f}°C")
        with col3:
            st.metric("Total Precipitation", f"{df['precipitation'].sum():.1f} mm")
        
        # Create chart
        fig = px.line(df, x="time", y="temperature", title="Temperature Forecast")
        st.plotly_chart(fig, use_container_width=True)
'''

with open('api_dashboard.py', 'w') as f:
    f.write(code.strip())

print("Created api_dashboard.py")
print("Run with: streamlit run api_dashboard.py")

## Module Summary

### What We've Learned:

1. **Real-Time Data Updates**
   - Auto-refresh mechanisms
   - Live data streaming
   - Progress indicators

2. **Download Functionality**
   - CSV, Excel, JSON exports
   - Formatted reports
   - Filtered data downloads

3. **User Authentication**
   - Login systems
   - Role-based access
   - Session management

4. **Responsive Design**
   - Mobile optimization
   - Flexible layouts
   - Touch-friendly controls

5. **Database/API Integration**
   - SQLite integration
   - API connections
   - Data caching

### Key Takeaways:
- Advanced features make dashboards production-ready
- Real-time updates enhance user experience
- Proper authentication protects sensitive data
- Responsive design ensures accessibility
- Integration capabilities connect to real data sources

## Practice Exercises

### Exercise 1: Build a Real-Time Monitor
Create a dashboard that:
- Monitors system metrics
- Updates every 5 seconds
- Shows alerts for thresholds
- Logs data to database

### Exercise 2: Create a Secure Dashboard
Build an app with:
- Multi-user authentication
- Role-based permissions
- Audit logging
- Session timeout

### Exercise 3: Design a Mobile Dashboard
Create a responsive dashboard that:
- Works on phones and tablets
- Has touch-friendly controls
- Adapts layout to screen size
- Optimizes data display

### Exercise 4: Integrate External Data
Build a dashboard that:
- Connects to a REST API
- Stores data in database
- Implements caching
- Handles errors gracefully

In [ ]:
# Space for your practice code
# Create advanced features for your dashboards

## Additional Resources

### Documentation
- [Streamlit API Reference](https://docs.streamlit.io/library/api-reference)
- [Plotly Documentation](https://plotly.com/python/)
- [SQLite Tutorial](https://www.sqlite.org/docs.html)
- [REST API Best Practices](https://restfulapi.net/)

### Advanced Topics
- [Streamlit Components](https://streamlit.io/components)
- [Custom Themes](https://docs.streamlit.io/library/advanced-features/theming)
- [Performance Optimization](https://docs.streamlit.io/library/advanced-features/caching)
- [Deployment Options](https://docs.streamlit.io/streamlit-community-cloud/get-started)

---

**Congratulations!** You've completed the Advanced Dashboard Features module. You now have the skills to build professional, production-ready dashboards with Streamlit!